In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.model_selection import StratifiedKFold

In [3]:
import py_entitymatching as em
A = em.read_csv_metadata('cv1.csv', key='ID')#Files before cleaning
B = em.read_csv_metadata('cv2.csv', key='ID')

No handlers could be found for logger "py_entitymatching.io.parsers"


In [3]:
ob = em.OverlapBlocker()
C1 = ob.block_tables(A, B, 'Name', 'Name', word_level=True, overlap_size=3, 
                    l_output_attrs=list(A.columns), 
                    r_output_attrs=list(B.columns),
                    show_progress=False)
C3 = ob.block_candset(C1, 'Resolution', 'Resolution',overlap_size=4,show_progress=False)

In [51]:
C3

,_id,ltable_ID,rtable_ID,ltable_3.5mm jack,ltable_Battery,ltable_Clock Speed,ltable_Card slot,ltable_GPS Support,ltable_GPU,ltable_Internal Storage,...,rtable_Name,rtable_OS,rtable_Price,rtable_Primary Camera,rtable_Clock Speed,rtable_SIM,rtable_Secondary Camera,rtable_Sensors,rtable_Weight,rtable_Resolution
0,0,1926,1,Yes,5000.0,Quad-core 1.4 GHz Cortex-A53,"microSD, up to 128 GB",Yes,Adreno 308,"16/32 GB, 2 GB RAM",...,"Moto E4 Plus (Fine Gold, 32 GB) (3 GB RAM)",Android Nougat 7.1.1,"��9,999",13MP,1.3 GHz,Nano SIM and Nano SIM,5MP,"Fingerprint, Accelerometer, Ambient Light, Proximity Sensor",198 g,1280 x 720 Pixels
1,1,2124,1,Yes,5000.0,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 128 GB (dedicated slot)",Yes,Mali-T720,"16/32 GB, 3 GB RAM",...,"Moto E4 Plus (Fine Gold, 32 GB) (3 GB RAM)",Android Nougat 7.1.1,"��9,999",13MP,1.3 GHz,Nano SIM and Nano SIM,5MP,"Fingerprint, Accelerometer, Ambient Light, Proximity Sensor",198 g,1280 x 720 Pixels
33,33,2121,21,Yes,2000.0,Quad-core 1.6 GHz,"microSD, up to 64 GB (dedicated slot)",Yes,NaN,"8 GB (2.8 GB user available), 2 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
41,41,2691,21,Yes,2800.0,Quad-core 1.5 GHz (SM-G750F)Quad-core 1.2 GHz Cortex-A53 (SM-G7508),"microSD, up to 64 GB (dedicated slot)",Yes,Mali-400MP4 (SM-G750F)Adreno 306 (SM-G7508),"16 GB, 1.5 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
48,48,905,21,Yes,3300.0,Octa-core (4x1.6 GHz Cortex-A53 & 4x1.0 GHz Cortex-A53)Octa-core 1.6 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",Yes,Adreno 405Mali-T830 MP1,"16 GB, 2 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
49,49,2954,21,Yes,2600.0,Quad-core 1.2 GHz Cortex-A7,"microSD, up to 64 GB (dedicated slot)",Yes,Adreno 305,"8 GB, 1.5 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
53,53,1084,21,Yes,3000.0,Octa-core 1.6 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",Yes,Mali-T830 MP1,"16 GB, 2 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
55,55,3194,21,Yes,3300.0,Octa-core 2.2 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",Yes,Adreno 506,"16 GB, 2 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
59,59,1344,21,Yes,3000.0,Octa-core (4x1.4 GHz Cortex-A53 & 4x1.0 GHz Cortex-A53)Octa-core 1.5 GHz Cortex-A53,"microSD, up to 256 GB (dedicated slot)",Yes,Adreno 405Mali-T720MP2,"16 GB, 1.5 GB RAM",...,"Samsung Galaxy J7 Nxt (Gold, 16 GB) (2 GB RAM)",Android Nougat 7.0,"��9,990",13MP,1.6 GHz,Micro SIM (3FF),5MP,"Accelerometer, Proximity Sensor",170 g,1280 x 720 Pixels
64,64,1326,22,Yes,2500.0,Quad-core 1.3 GHz Cortex-A53,"microSD, up to 32 GB",Yes,Mali-T720,"16 GB, 3 GB RAM",...,"Micromax Bharat 4 (Black/Jet Black, 16 GB) (1 GB RAM)",Android Nougat 7,"��4,999",5MP,1.3 GHz,Micro SIM and Micro SIM,5MP,"Gravity Sensors, Proximity Sensor (By SW)",150 g,1280 x 720 Pixels


In [4]:
S = em.sample_table(C3, 300)
S.to_csv('sampledcross.csv')

In [23]:
#labeled_data1.csv has the labeled data corresponding to sampledcross.csv
l_data = em.read_csv_metadata('labeled_data1.csv', key='_id', 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                         ltable=A, rtable=B)
match_f = em.get_features_for_matching(A, B)
H = em.extract_feature_vecs(l_data, feature_table=match_f,attrs_before = None, attrs_after=['Class'])
H.fillna(value=0, inplace=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [24]:
train_test = em.split_train_test(H, train_proportion=0.7)
train_set = train_test['train']
test_set = train_test['test']
train_set.to_csv('training_split')
test_set.to_csv('test_split')
train_labels = train_set['Class']
test_labels = test_set['Class']
del train_set['Class']

In [25]:
dt = em.DTMatcher(max_depth=5)
rf= em.RFMatcher()
nb= em.NBMatcher()
lg= em.LogRegMatcher()
ln= em.LinRegMatcher()
svm= em.SVMMatcher()
matchers = [dt,rf,nb,lg, ln, svm] 
before_cleaning = em.select_matcher(matchers, x=train_set, y=train_labels, table=None, exclude_attrs=None, target_attr=None, metric_to_select_matcher='precision', metrics_to_display=['precision', 'recall', 'f1'], k=5, n_jobs=-1, random_state=None)

In [27]:
before_cleaning

OrderedDict([('selected_matcher',
              <py_entitymatching.matcher.rfmatcher.RFMatcher at 0x1d901710>),
             ('cv_stats',
                                                 Matcher  Average precision  Average recall  \
              0        DecisionTree_23299999200727804251           0.805556        0.812634   
              1        RandomForest_23299999200727804251           0.852525        0.794762   
              2          NaiveBayes_23299999200727804251           0.631375        0.818333   
              3  LogisticRegression_23299999200727804251           0.839524        0.811429   
              4    LinearRegression_23299999200727804251           0.747619        0.676667   
              5                 SVM_23299999200727804251           0.700000        0.000000   
              
                 Average f1  
              0    0.742749  
              1    0.743875  
              2    0.704381  
              3    0.745065  
              4    0.643649  
  

In [ ]:
A = em.read_csv_metadata('C:\Python27amd64\\new1_1.csv', key='ID')#Files after cleaning
B = em.read_csv_metadata('C:\Python27amd64\\new2_1.csv', key='ID')
ob = em.OverlapBlocker()
C1 = ob.block_tables(A, B, 'Name', 'Name', word_level=True, overlap_size=3, 
                    l_output_attrs=list(A.columns), 
                    r_output_attrs=list(B.columns),
                    show_progress=False)
C3 = ob.block_candset(C1, 'Resolution', 'Resolution',overlap_size=4,show_progress=False)
S = em.sample_table(C3, 300)
S.to_csv('sampled.csv')

In [28]:
#labeled_data.csv has the labeled data corresponding to sampled.csv
l_data = em.read_csv_metadata('labeled_data.csv', key='_id', 
                         fk_ltable='ltable_ID', fk_rtable='rtable_ID',
                         ltable=A, rtable=B)

match_f = em.get_features_for_matching(A, B)
H = em.extract_feature_vecs(l_data, feature_table=match_f,attrs_before = None, attrs_after=['Class'])
H.fillna(value=0, inplace=True)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,numeric,numeric,Exact Match; Absolute Norm
1,3.5mm jack,3.5mm jack,short string (1 word to 5 words),short string (1 word),Not Applicable: Types do not match
2,Battery,Battery,numeric,numeric,Exact Match; Absolute Norm
3,Clock Speed,Clock Speed,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
4,Card slot,Card slot,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
5,GPS Support,GPS Support,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
6,GPU,GPU,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Internal Storage,Internal Storage,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
8,NFC,NFC,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
9,Name,Name,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"


Do you want to proceed? (y/n):y


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


In [29]:
train_test = em.split_train_test(H, train_proportion=0.7)
train_set = train_test['train']
test_set = train_test['test']
train_set.to_csv('training_split_after_cleaning')
test_set.to_csv('test_split_after_cleaning')
train_labels = train_set['Class']
test_labels = test_set['Class']
del train_set['Class']

In [30]:
dt = em.DTMatcher(max_depth=5)
rf= em.RFMatcher()
nb= em.NBMatcher()
lg= em.LogRegMatcher()
ln= em.LinRegMatcher()
svm= em.SVMMatcher()
#xg = em.XGBoostMatcher()
matchers = [dt,rf,nb,lg, ln, svm] 
after_cleaning = em.select_matcher(matchers, x=train_set, y=train_labels, table=None, exclude_attrs=None, target_attr=None, metric_to_select_matcher='precision', metrics_to_display=['precision', 'recall', 'f1'], k=5, n_jobs=-1, random_state=None)

In [31]:
after_cleaning

OrderedDict([('selected_matcher',
              <py_entitymatching.matcher.rfmatcher.RFMatcher at 0x1dbd06a0>),
             ('cv_stats',
                                                 Matcher  Average precision  Average recall  \
              0        DecisionTree_04600007272948804251           0.897619        0.763095   
              1        RandomForest_04600007272948804251           0.971429        0.854902   
              2          NaiveBayes_04600007272948804251           0.797172        0.899231   
              3  LogisticRegression_04600007272948804251           0.875065        0.897483   
              4    LinearRegression_04600007272948804251           0.652887        0.688254   
              5                 SVM_04600007272948804251           0.400000        0.117436   
              
                 Average f1  
              0    0.790476  
              1    0.847379  
              2    0.836748  
              3    0.863333  
              4    0.746017  
  

In [39]:
train_set.insert(loc = len(train_set.columns),column='Class', value=list(train_labels))

In [42]:
rf.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = rf.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')

In [43]:
eval_summary

OrderedDict([('prec_numerator', 24.0),
             ('prec_denominator', 26.0),
             ('precision', 0.9230769230769231),
             ('recall_numerator', 24.0),
             ('recall_denominator', 31.0),
             ('recall', 0.7741935483870968),
             ('f1', 0.8421052631578947),
             ('pred_pos_num', 26.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls', [(3175L, 1474L), (1106L, 636L)]),
             ('pred_neg_num', 64.0),
             ('false_neg_num', 7.0),
             ('false_neg_ls',
              [(2591L, 1078L),
               (3025L, 123L),
               (1308L, 494L),
               (2150L, 355L),
               (2713L, 1493L),
               (2038L, 830L),
               (2710L, 80L)])])

In [44]:
del test_set['Predicted_Labels']
svm.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = svm.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')

In [45]:
eval_summary

OrderedDict([('prec_numerator', 16.0),
             ('prec_denominator', 16.0),
             ('precision', 1.0),
             ('recall_numerator', 16.0),
             ('recall_denominator', 31.0),
             ('recall', 0.5161290322580645),
             ('f1', 0.6808510638297872),
             ('pred_pos_num', 16.0),
             ('false_pos_num', 0.0),
             ('false_pos_ls', []),
             ('pred_neg_num', 74.0),
             ('false_neg_num', 15.0),
             ('false_neg_ls',
              [(2274L, 358L),
               (2600L, 766L),
               (2713L, 1493L),
               (1289L, 677L),
               (1932L, 1308L),
               (3025L, 123L),
               (2292L, 231L),
               (392L, 1308L),
               (920L, 205L),
               (1289L, 970L),
               (2150L, 355L),
               (2591L, 1078L),
               (2108L, 80L),
               (2038L, 830L),
               (2710L, 80L)])])

In [46]:
del test_set['Predicted_Labels']
nb.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = nb.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')
eval_summary

OrderedDict([('prec_numerator', 29.0),
             ('prec_denominator', 31.0),
             ('precision', 0.9354838709677419),
             ('recall_numerator', 29.0),
             ('recall_denominator', 31.0),
             ('recall', 0.9354838709677419),
             ('f1', 0.9354838709677419),
             ('pred_pos_num', 31.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls', [(3175L, 1474L), (1106L, 636L)]),
             ('pred_neg_num', 59.0),
             ('false_neg_num', 2.0),
             ('false_neg_ls', [(1289L, 970L), (1289L, 677L)])])

In [47]:
lg= em.LogRegMatcher()
del test_set['Predicted_Labels']
lg.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = lg.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')
eval_summary

OrderedDict([('prec_numerator', 24.0),
             ('prec_denominator', 27.0),
             ('precision', 0.8888888888888888),
             ('recall_numerator', 24.0),
             ('recall_denominator', 31.0),
             ('recall', 0.7741935483870968),
             ('f1', 0.8275862068965517),
             ('pred_pos_num', 27.0),
             ('false_pos_num', 3.0),
             ('false_pos_ls', [(540L, 1068L), (1106L, 636L), (1344L, 573L)]),
             ('pred_neg_num', 63.0),
             ('false_neg_num', 7.0),
             ('false_neg_ls',
              [(2600L, 766L),
               (2713L, 1493L),
               (1932L, 1308L),
               (392L, 1308L),
               (1289L, 970L),
               (2150L, 355L),
               (1289L, 677L)])])

In [48]:
ln= em.LinRegMatcher()
del test_set['Predicted_Labels']
ln.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = ln.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')
eval_summary

OrderedDict([('prec_numerator', 21.0),
             ('prec_denominator', 24.0),
             ('precision', 0.875),
             ('recall_numerator', 21.0),
             ('recall_denominator', 31.0),
             ('recall', 0.6774193548387096),
             ('f1', 0.7636363636363636),
             ('pred_pos_num', 24.0),
             ('false_pos_num', 3.0),
             ('false_pos_ls', [(3127L, 231L), (540L, 1068L), (1106L, 636L)]),
             ('pred_neg_num', 66.0),
             ('false_neg_num', 10.0),
             ('false_neg_ls',
              [(2274L, 358L),
               (2600L, 766L),
               (2713L, 1493L),
               (1289L, 677L),
               (3025L, 123L),
               (392L, 1308L),
               (1289L, 970L),
               (2150L, 355L),
               (2591L, 1078L),
               (2038L, 830L)])])

In [49]:
del test_set['Predicted_Labels']
dt.fit(table=train_set, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'], target_attr='Class')
pred_table = dt.predict(table=test_set,  exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'Class'],  append=True, target_attr='Predicted_Labels')
eval_summary = em.eval_matches(pred_table, 'Class', 'Predicted_Labels')
eval_summary

OrderedDict([('prec_numerator', 26.0),
             ('prec_denominator', 27.0),
             ('precision', 0.9629629629629629),
             ('recall_numerator', 26.0),
             ('recall_denominator', 31.0),
             ('recall', 0.8387096774193549),
             ('f1', 0.896551724137931),
             ('pred_pos_num', 27.0),
             ('false_pos_num', 1.0),
             ('false_pos_ls', [(3175L, 1474L)]),
             ('pred_neg_num', 63.0),
             ('false_neg_num', 5.0),
             ('false_neg_ls',
              [(1289L, 970L),
               (392L, 1308L),
               (1289L, 677L),
               (2150L, 355L),
               (2710L, 80L)])])